# Voting classifier

In [1]:
from sklearn.ensemble import VotingClassifier

In [11]:
# load basic libraries
import pandas as pd
import numpy as np
import mne
from pathlib import Path
import pickle
import time

%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt

from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, cross_val_score

# set directories
# %cd D:/Programy/Anaconda3/Projects/EEG ML project # working directory
%cd D:
pkls = './Pickles/' # objects & variables

D:\Programy\Anaconda3\Projects\EEG ML project


In [3]:
# load split sets, eeg only
with open(pkls +'xy_train.pkl', 'rb') as handle:
    x_train = pickle.load(handle)
    y_train = pickle.load(handle)
y_train.shape
x_train.shape

(6040, 94)

In [6]:
# load models to include in the voter

with open(pkls + 'best_svc.pkl', 'rb') as handle:
    best_svc = pickle.load(handle)
    best_svc_res = pickle.load(handle)
    
with open(pkls + 'best_knn.pkl', 'rb') as handle:
    best_knn = pickle.load(handle)
    best_knn_res = pickle.load(handle)
    
with open(pkls + 'best_gnb.pkl', 'rb') as handle:
    best_gnb = pickle.load(handle)
    best_gnb_res = pickle.load(handle)
    
with open(pkls + 'best_extr.pkl', 'rb') as handle:
    best_extr = pickle.load(handle)
    best_extr_res = pickle.load(handle)
    
with open(pkls + 'best_forest.pkl', 'rb') as handle:
    best_forest = pickle.load(handle)
    best_forest_res = pickle.load(handle)

In [13]:
voting_clf = VotingClassifier(
    estimators=[('gnb', best_gnb), 
                ('knn', best_knn), 
                ('svc', best_svc),
                ('extr', best_extr),
                ('forest', best_forest)],
    voting='hard')

voting_clf.fit(x_train, y_train.values.ravel())

VotingClassifier(estimators=[('gnb',
                              GaussianNB(priors=None, var_smoothing=1e-09)),
                             ('knn',
                              KNeighborsClassifier(algorithm='auto',
                                                   leaf_size=30,
                                                   metric='manhattan',
                                                   metric_params=None,
                                                   n_jobs=None, n_neighbors=19,
                                                   p=2, weights='uniform')),
                             ('svc',
                              SVC(C=1, break_ties=False, cache_size=200,
                                  class_weight=None, coef0=0.0,
                                  decision_function_shape='ovr', degree=3,
                                  gamma=0....
                                                     criterion='gini',
                                                  

In [14]:
# evaluate
cv = 10
scores = cross_val_score(voting_clf, x_train, y_train.values.ravel(), cv=cv)
scoresf1 = cross_val_score(voting_clf, x_train, y_train.values.ravel(), cv=cv, scoring='f1_weighted')

voter_res = pd.DataFrame(data = [round(scores.mean(),3), round(scores.std() * 2, 3), 
                                 round(scoresf1.mean(),3), round(scoresf1.std() * 2,3)]).T
voter_res.columns = ['acc mean', 'acc 2sd', 'f1 mean','f1 2sd']
voter_res.index = ['voter']
voter_res

,acc mean,acc 2sd,f1 mean,f1 2sd
voter,0.527,0.079,0.507,0.057


In [12]:
# train set score for comparison
voting_clf.fit(x_train, y_train.values.ravel())
print("training acc mean: {:.4f}".format(
    voting_clf.score(x_train, y_train)))
print("training f1  mean: {:.4f}".format(
    f1_score(y_train, voting_clf.predict(x_train), average = 'weighted')))

training acc mean: 0.7248
training f1  mean: 0.7208


In [15]:
with open('./Pickles/voter.pkl', 'wb') as handle:
    pickle.dump(voting_clf, handle)
    pickle.dump(voter_res, handle)